In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
pre = pd.read_excel('EP42_45degree_FFP.xlsx')
next = pd.read_excel('Epix42_PR_45degree.xlsx')

In [46]:
pre.sample(3)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
103,17803,Epix_Pro_FT2,17,外部電流(discharging)|VEXT Current(discharging),2/404,0.50
178,17798,Epix_Pro_AutoATE,6,BAT3.3V開機電流|BAT3.3V startup current,1/570,0.18
183,17798,Epix_Pro_AutoATE,61,VOVSS - High,1/570,0.18


In [47]:
next.sample(3)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
120,17801,Epix_Pro_HT,47,電池溫度|Battery Temperature non_recheck,4/619,0.65
100,17859,Epix_Pro_ATE_TI,53,D105 – 540nm Brightness (20mA) Measure diff wi...,7/765,0.92
202,17822,Epix_Pro_ALS,7,ALS Luminance (WLED ON),1/562,0.18


# Tweak and Filter

Blacklist

In [48]:
ProcessTypeBlackList = 'Pack|Click|RabbitCard|EasyCard|DeleteBundle|ProdScan|FileCopyer'
ItemNameBlackList = 'ESN|Check ProductionMap|Fixture ID'

tweak_filter def

In [49]:
def tweak(raw):
    return(raw
    .drop('Comment',axis=1, errors='ignore')
    .rename(columns = lambda name : name.replace(" ",'').replace('/','').replace('%',''))
    .sort_values('Retest', ascending=False)
    .astype({'ItemNameType':'category','ProcessType':'category','Item':'category','ItemName':'category'})
    .assign(CountESN = lambda df : df.CountCountESN.str.split('/').str[1].astype('int16'))
    .query("~ProcessType.str.contains(@ProcessTypeBlackList) and ~ItemName.str.contains(@ItemNameBlackList)")
    )
def tweak_filter(raw):
    return(raw
    .pipe(tweak)
    .query("CountESN > CountESN.quantile(0.05)")
    .query("Retest >= Retest.mean()")
    )

# Merge two phases

In [50]:
pre_next = pd.merge(
    left=tweak_filter(pre), right=tweak(next), how='left',
    left_on=['ItemNameType','ProcessType','Item','ItemName'], right_on=['ItemNameType','ProcessType','Item','ItemName'],
    suffixes=['_pre','_next'])

In [51]:
pre_next.to_excel('output.xlsx', index=False)

# RetryImprove def

In [52]:
def RetryImprove(df):
    return((100 * (df.Retest_pre.sum() - df.Retest_next.sum()) / df.Retest_pre.sum()).round(3))

In [53]:
RetryImprove(pre_next)

60.718

In [55]:
def RetryDiff(df):
    return(df
     .assign(diff = df.Retest_pre - df.Retest_next)
     [['ProcessType','ItemName','diff']]
     .sort_values('diff', ascending=False)
     .dropna(axis=0)
     .style.bar('diff', color=['red','green'])
     )
RetryDiff(pre_next)

,ProcessType,ItemName,diff
0,Epix_Pro_ECG_FT,ECG impedance short,15.880000
2,Epix_Pro_ATE_TI,AFE Power Consumption data,14.450000
3,Epix_Pro_ATE_TI,Temperature Sensor Temperature,12.820000
5,Epix_Pro_Gyro,電池電壓|Battery Voltage,6.660000
24,Epix_Pro_FT2,Power Box Sampling BL on normal(Avg) diff with ATE,1.750000
11,Epix_Pro_FT1,GPS_ (L1 + L5) by ref,1.270000
18,Epix_Pro_Gyro,Check FT3 Test Data,1.170000
20,Epix_Pro_FT2,ANT_power 2450MHz,0.880000
17,Epix_Pro_ATE_TI,AFE Power Consumption Test,0.670000
15,Epix_Pro_FT1,006J Comparison,-0.300000


# Total improvement rate

In [54]:
100*((tweak(pre).Retest.sum() - tweak(next).Retest.sum())/ tweak(pre).Retest.sum())

14.99408434040396